In [1]:
import tensorflow as tf
import numpy as np

/Users/ljy/miniconda2/envs/xgb/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [58]:
class TCNNConfig(object):
    """CNN配置参数"""
    embedding_size = 64  # 词向量维度
    seq_length = 600  # 序列长度
    num_classes = 10  # 类别数
    num_filters = 256  # 卷积核数目
    kernel_size = 5  # 卷积核尺寸
    vocab_size = 5000  # 词汇表大小

    hidden_dim = 128  # 全连接层神经元

    dropout_keep_prob = 0.5  # dropout保留比例
    learning_rate = 1e-3  # 学习率

    batch_size = 64  # 每批训练大小
    num_epochs = 10  # 总迭代轮次

    print_per_batch = 100  # 每多少轮输出一次结果
    save_per_batch = 10  # 每多少轮存入tensorboard

    l2_reg_lambda = 0.1

In [59]:
# 实现思路：
# 将对x的标签化放在模型中完成，只将对y的label化放在外面。个人倾向于都放在模型中，后续进行修改
# embedding - cnn - maxpooling - fc - dropout - classify - result
class TextCNN(object):
    def __init__(self, config):
        self.config = config
        
        # placrholders for three inputs
        self.input_x = tf.placeholder(tf.int32, [None, self.config.seq_length], name='input_x')
        self.input_y = tf.placeholder(tf.float32, [None, self.config.num_classes], name='input_y')
        self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')
        self.l2_loss = tf.constant(0.0)
        
        # init cnn model different the original form, i prefer this one
        self.cnn()
    def cnn(self, ):
        # embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            # w is the embedding matrix thar we learn during training and use randomuniform distribution to initialize it
            W = tf.Variable(tf.random_uniform([self.config.vocab_size, self.config.embedding_size], -1, 1), name = "embedding_W")
            embedded_ouput = tf.nn.embedding_lookup(W, self.input_x)
            print(embedded_ouput)
            
        # define the cnn layer with max pooling
        # use 2d input instead of 4d so we could jsut conv1d, simplify the calculation of dimentsions
        with tf.name_scope('cnn'):
            # CNN layer
            conv = tf.layers.conv1d(embedded_ouput, self.config.num_filters, self.config.kernel_size, name='cnn_conv')
            # global max pooling layer
            # just get the max one on dimension 1
            gmp = tf.reduce_max(conv, reduction_indices=[1], name='gmp')

        # add a fully-connect with a dropout
        
        with tf.name_scope("dropout", ):
            fc = tf.layers.dense(gmp, self.config.hidden_dim, name='fc1')
            fc = tf.contrib.layers.dropout(fc, self.keep_prob)
            fc = tf.nn.relu(fc)
            
        with tf.name_scope("score", ):
            W = tf.get_variable(
                "W",
                shape=[self.config.hidden_dim, self.config.num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[self.config.num_classes]), name="b")
            self.l2_loss += tf.nn.l2_loss(W)
            self.l2_loss += tf.nn.l2_loss(b)
            
            self.logits = tf.nn.xw_plus_b(fc, W, b, name="scores")
            self.y_pred_cls = tf.argmax(tf.nn.softmax(self.logits), 1)  # 预测类别
        
        with tf.name_scope("loss",):
            # loss function cross_entropy
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.input_y)
            self.loss = tf.reduce_mean(cross_entropy) + self.config.l2_reg_lambda * self.l2_loss
            
            self.optim = tf.train.AdamOptimizer(learning_rate=self.config.learning_rate).minimize(self.loss)
        
        with tf.name_scope("accuracy", ):
            # 准确率
            correct_predictions = tf.equal(tf.argmax(self.input_y, 1), self.y_pred_cls)
            self.acc = tf.reduce_mean(tf.cast(correct_predictions, "float"), name = "accuracy")

            
        

In [60]:
tf.reset_default_graph() # 添加这句的目的是在jupyter notebook里面清楚缓存，不然会出现变量重用的问题
print('Configuring CNN model...')
config = TCNNConfig()
if not os.path.exists(vocab_dir):  # 如果不存在词汇表，重建
    build_vocab(train_dir, vocab_dir, config.vocab_size)
categories, cat_to_id = read_category()
words, word_to_id = read_vocab(vocab_dir)
config.vocab_size = len(words)
model = TextCNN(config)
train()

Configuring CNN model...
Tensor("embedding/embedding_lookup:0", shape=(?, 600, 64), dtype=float32, device=/device:CPU:0)
Configuring TensorBoard and Saver...
Loading training and validation data...
Time usage: 0:00:13
Training and evaluating...
Epoch: 1
Iter:      0, Train Loss:    3.7, Train Acc:  10.94%, Val Loss:    3.7, Val Acc:   9.34%, Time: 0:00:06 *
Iter:    100, Train Loss:    2.1, Train Acc:  64.06%, Val Loss:    2.3, Val Acc:  49.60%, Time: 0:00:58 *
Iter:    200, Train Loss:    0.9, Train Acc:  85.94%, Val Loss:    1.2, Val Acc:  73.74%, Time: 0:01:49 *
Iter:    300, Train Loss:   0.69, Train Acc:  81.25%, Val Loss:   0.93, Val Acc:  76.42%, Time: 0:02:40 *
Iter:    400, Train Loss:   0.53, Train Acc:  92.19%, Val Loss:   0.74, Val Acc:  81.94%, Time: 0:03:32 *
Iter:    500, Train Loss:   0.37, Train Acc:  96.88%, Val Loss:   0.63, Val Acc:  86.04%, Time: 0:04:23 *
Iter:    600, Train Loss:   0.28, Train Acc:  95.31%, Val Loss:   0.54, Val Acc:  87.64%, Time: 0:05:15 *
Iter

In [61]:
test() # compared with the original chinese one, it is improved by 1.51% 94.12%

Loading test data...
INFO:tensorflow:Restoring parameters from checkpoints/textcnn/best_validation
Testing...
Test Loss:    0.2, Test Acc:  95.63%
Precision, Recall and F1-Score...
             precision    recall  f1-score   support

         体育       0.99      0.99      0.99      1000
         财经       0.96      0.99      0.97      1000
         房产       1.00      1.00      1.00      1000
         家居       0.96      0.88      0.92      1000
         教育       0.90      0.93      0.91      1000
         科技       0.91      0.98      0.94      1000
         时尚       0.95      0.97      0.96      1000
         时政       0.95      0.92      0.93      1000
         游戏       0.99      0.94      0.96      1000
         娱乐       0.97      0.97      0.97      1000

avg / total       0.96      0.96      0.96     10000

Confusion Matrix...
[[992   0   0   1   2   3   0   2   0   0]
 [  0 990   0   0   1   2   0   7   0   0]
 [  0   0 996   1   2   1   0   0   0   0]
 [  3  14   1 882  18  35  14  

In [49]:
## train in jupyter
from __future__ import print_function
import os
import sys
import time
from datetime import timedelta

import numpy as np
import tensorflow as tf
from sklearn import metrics
from data.cnews_loader import read_vocab, read_category, batch_iter, process_file, build_vocab


# 这个代码拆解了很多，最后才进行了训练
base_dir = 'data/cnews'
train_dir = os.path.join(base_dir, 'cnews.train.txt')
test_dir = os.path.join(base_dir, 'cnews.test.txt')
val_dir = os.path.join(base_dir, 'cnews.val.txt')
vocab_dir = os.path.join(base_dir, 'cnews.vocab.txt')

save_dir = 'checkpoints/textcnn'
save_path = os.path.join(save_dir, 'best_validation')  # 最佳验证结果保存路径


def get_time_dif(start_time):
    """获取已使用时间"""
    end_time = time.time()
    time_dif = end_time - start_time
    return timedelta(seconds=int(round(time_dif)))


def feed_data(x_batch, y_batch, keep_prob):
    feed_dict = {
        model.input_x: x_batch,
        model.input_y: y_batch,
        model.keep_prob: keep_prob
    }
    return feed_dict


def evaluate(sess, x_, y_):
    """评估在某一数据上的准确率和损失"""
    data_len = len(x_)
    batch_eval = batch_iter(x_, y_, 128)
    total_loss = 0.0
    total_acc = 0.0
    for x_batch, y_batch in batch_eval:
        batch_len = len(x_batch)
        feed_dict = feed_data(x_batch, y_batch, 1.0)
        loss, acc = sess.run([model.loss, model.acc], feed_dict=feed_dict)
        total_loss += loss * batch_len
        total_acc += acc * batch_len

    return total_loss / data_len, total_acc / data_len


def train():
    print("Configuring TensorBoard and Saver...")
    # 配置 Tensorboard，重新训练时，请将tensorboard文件夹删除，不然图会覆盖
    tensorboard_dir = 'tensorboard/textcnn'
    if not os.path.exists(tensorboard_dir):
        os.makedirs(tensorboard_dir)

    tf.summary.scalar("loss", model.loss)
    tf.summary.scalar("accuracy", model.acc)
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter(tensorboard_dir)

    # 配置 Saver
    saver = tf.train.Saver()
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    print("Loading training and validation data...")
    # 载入训练集与验证集
    start_time = time.time()
    x_train, y_train = process_file(train_dir, word_to_id, cat_to_id, config.seq_length)
    x_val, y_val = process_file(val_dir, word_to_id, cat_to_id, config.seq_length)
    time_dif = get_time_dif(start_time)
    print("Time usage:", time_dif)

    # 创建session
    session = tf.Session()
    session.run(tf.global_variables_initializer())
    writer.add_graph(session.graph)

    print('Training and evaluating...')
    start_time = time.time()
    total_batch = 0  # 总批次
    best_acc_val = 0.0  # 最佳验证集准确率
    last_improved = 0  # 记录上一次提升批次
    require_improvement = 1000  # 如果超过1000轮未提升，提前结束训练

    flag = False
    for epoch in range(config.num_epochs):
        print('Epoch:', epoch + 1)
        batch_train = batch_iter(x_train, y_train, config.batch_size)
        for x_batch, y_batch in batch_train:
            feed_dict = feed_data(x_batch, y_batch, config.dropout_keep_prob)

            if total_batch % config.save_per_batch == 0:
                # 每多少轮次将训练结果写入tensorboard scalar
                s = session.run(merged_summary, feed_dict=feed_dict)
                writer.add_summary(s, total_batch)

            if total_batch % config.print_per_batch == 0:
                # 每多少轮次输出在训练集和验证集上的性能
                feed_dict[model.keep_prob] = 1.0
                loss_train, acc_train = session.run([model.loss, model.acc], feed_dict=feed_dict)
                loss_val, acc_val = evaluate(session, x_val, y_val)  # todo

                if acc_val > best_acc_val:
                    # 保存最好结果
                    best_acc_val = acc_val
                    last_improved = total_batch
                    saver.save(sess=session, save_path=save_path)
                    improved_str = '*'
                else:
                    improved_str = ''

                time_dif = get_time_dif(start_time)
                msg = 'Iter: {0:>6}, Train Loss: {1:>6.2}, Train Acc: {2:>7.2%},' \
                      + ' Val Loss: {3:>6.2}, Val Acc: {4:>7.2%}, Time: {5} {6}'
                print(msg.format(total_batch, loss_train, acc_train, loss_val, acc_val, time_dif, improved_str))

            feed_dict[model.keep_prob] = config.dropout_keep_prob
            session.run(model.optim, feed_dict=feed_dict)  # 运行优化
            total_batch += 1

            if total_batch - last_improved > require_improvement:
                # 验证集正确率长期不提升，提前结束训练
                print("No optimization for a long time, auto-stopping...")
                flag = True
                break  # 跳出循环
        if flag:  # 同上
            break


def test():
    print("Loading test data...")
    start_time = time.time()
    x_test, y_test = process_file(test_dir, word_to_id, cat_to_id, config.seq_length)

    session = tf.Session()
    session.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.restore(sess=session, save_path=save_path)  # 读取保存的模型

    print('Testing...')
    loss_test, acc_test = evaluate(session, x_test, y_test)
    msg = 'Test Loss: {0:>6.2}, Test Acc: {1:>7.2%}'
    print(msg.format(loss_test, acc_test))

    batch_size = 128
    data_len = len(x_test)
    num_batch = int((data_len - 1) / batch_size) + 1

    y_test_cls = np.argmax(y_test, 1)
    y_pred_cls = np.zeros(shape=len(x_test), dtype=np.int32)  # 保存预测结果
    for i in range(num_batch):  # 逐批次处理
        start_id = i * batch_size
        end_id = min((i + 1) * batch_size, data_len)
        feed_dict = {
            model.input_x: x_test[start_id:end_id],
            model.keep_prob: 1.0
        }
        y_pred_cls[start_id:end_id] = session.run(model.y_pred_cls, feed_dict=feed_dict)

    # 评估
    print("Precision, Recall and F1-Score...")
    print(metrics.classification_report(y_test_cls, y_pred_cls, target_names=categories))

    # 混淆矩阵
    print("Confusion Matrix...")
    cm = metrics.confusion_matrix(y_test_cls, y_pred_cls)
    print(cm)

    time_dif = get_time_dif(start_time)
    print("Time usage:", time_dif)





In [50]:
# main func
tf.reset_default_graph() # 添加这句的目的是在jupyter notebook里面清楚缓存，不然会出现变量重用的问题
print('Configuring CNN model...')
config = TCNNConfig()
if not os.path.exists(vocab_dir):  # 如果不存在词汇表，重建
    build_vocab(train_dir, vocab_dir, config.vocab_size)
categories, cat_to_id = read_category()
words, word_to_id = read_vocab(vocab_dir)
config.vocab_size = len(words)
model = TextCNN(config)
train()

Configuring CNN model...
Tensor("embedding/embedding_lookup:0", shape=(?, 600, 64), dtype=float32, device=/device:CPU:0)
Configuring TensorBoard and Saver...
Loading training and validation data...
Time usage: 0:00:19
Training and evaluating...
Epoch: 1
Iter:      0, Train Loss:    2.5, Train Acc:  10.94%, Val Loss:    2.5, Val Acc:   9.38%, Time: 0:00:07 *
Iter:    100, Train Loss:    1.6, Train Acc:  62.50%, Val Loss:    1.8, Val Acc:  49.92%, Time: 0:01:04 *
Iter:    200, Train Loss:    0.7, Train Acc:  78.12%, Val Loss:   0.94, Val Acc:  74.62%, Time: 0:02:00 *
Iter:    300, Train Loss:   0.27, Train Acc:  90.62%, Val Loss:   0.74, Val Acc:  78.36%, Time: 0:03:01 *
Iter:    400, Train Loss:   0.25, Train Acc:  95.31%, Val Loss:   0.51, Val Acc:  85.66%, Time: 0:04:00 *
Iter:    500, Train Loss:   0.18, Train Acc:  92.19%, Val Loss:   0.47, Val Acc:  86.90%, Time: 0:04:59 *
Iter:    600, Train Loss:    0.2, Train Acc:  90.62%, Val Loss:   0.46, Val Acc:  87.28%, Time: 0:06:00 *
Iter

KeyboardInterrupt: 

In [5]:
from __future__ import print_function

import os
import sys
import time
from datetime import timedelta

import numpy as np
import tensorflow as tf
from sklearn import metrics

from cnn_model import TCNNConfig, TextCNN
from data.cnews_loader import read_vocab, read_category, batch_iter, process_file, build_vocab


# 这个代码拆解了很多，最后才进行了训练
base_dir = 'data/cnews'
train_dir = os.path.join(base_dir, 'cnews.train.txt')
test_dir = os.path.join(base_dir, 'cnews.test.txt')
val_dir = os.path.join(base_dir, 'cnews.val.txt')
vocab_dir = os.path.join(base_dir, 'cnews.vocab.txt')

save_dir = 'checkpoints/textcnn'
save_path = os.path.join(save_dir, 'best_validation')  # 最佳验证结果保存路径


def get_time_dif(start_time):
    """获取已使用时间"""
    end_time = time.time()
    time_dif = end_time - start_time
    return timedelta(seconds=int(round(time_dif)))


def feed_data(x_batch, y_batch, keep_prob):
    feed_dict = {
        model.input_x: x_batch,
        model.input_y: y_batch,
        model.keep_prob: keep_prob
    }
    return feed_dict


def evaluate(sess, x_, y_):
    """评估在某一数据上的准确率和损失"""
    data_len = len(x_)
    batch_eval = batch_iter(x_, y_, 128)
    total_loss = 0.0
    total_acc = 0.0
    for x_batch, y_batch in batch_eval:
        batch_len = len(x_batch)
        feed_dict = feed_data(x_batch, y_batch, 1.0)
        loss, acc = sess.run([model.loss, model.acc], feed_dict=feed_dict)
        total_loss += loss * batch_len
        total_acc += acc * batch_len

    return total_loss / data_len, total_acc / data_len


def train():
    print("Configuring TensorBoard and Saver...")
    # 配置 Tensorboard，重新训练时，请将tensorboard文件夹删除，不然图会覆盖
    tensorboard_dir = 'tensorboard/textcnn'
    if not os.path.exists(tensorboard_dir):
        os.makedirs(tensorboard_dir)

    tf.summary.scalar("loss", model.loss)
    tf.summary.scalar("accuracy", model.acc)
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter(tensorboard_dir)

    # 配置 Saver
    saver = tf.train.Saver()
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    print("Loading training and validation data...")
    # 载入训练集与验证集
    start_time = time.time()
    x_train, y_train = process_file(train_dir, word_to_id, cat_to_id, config.seq_length)
    x_val, y_val = process_file(val_dir, word_to_id, cat_to_id, config.seq_length)
    time_dif = get_time_dif(start_time)
    print("Time usage:", time_dif)

    # 创建session
    session = tf.Session()
    session.run(tf.global_variables_initializer())
    writer.add_graph(session.graph)

    print('Training and evaluating...')
    start_time = time.time()
    total_batch = 0  # 总批次
    best_acc_val = 0.0  # 最佳验证集准确率
    last_improved = 0  # 记录上一次提升批次
    require_improvement = 1000  # 如果超过1000轮未提升，提前结束训练

    flag = False
    for epoch in range(config.num_epochs):
        print('Epoch:', epoch + 1)
        batch_train = batch_iter(x_train, y_train, config.batch_size)
        for x_batch, y_batch in batch_train:
            feed_dict = feed_data(x_batch, y_batch, config.dropout_keep_prob)

            if total_batch % config.save_per_batch == 0:
                # 每多少轮次将训练结果写入tensorboard scalar
                s = session.run(merged_summary, feed_dict=feed_dict)
                writer.add_summary(s, total_batch)

            if total_batch % config.print_per_batch == 0:
                # 每多少轮次输出在训练集和验证集上的性能
                feed_dict[model.keep_prob] = 1.0
                loss_train, acc_train = session.run([model.loss, model.acc], feed_dict=feed_dict)
                loss_val, acc_val = evaluate(session, x_val, y_val)  # todo

                if acc_val > best_acc_val:
                    # 保存最好结果
                    best_acc_val = acc_val
                    last_improved = total_batch
                    saver.save(sess=session, save_path=save_path)
                    improved_str = '*'
                else:
                    improved_str = ''

                time_dif = get_time_dif(start_time)
                msg = 'Iter: {0:>6}, Train Loss: {1:>6.2}, Train Acc: {2:>7.2%},' \
                      + ' Val Loss: {3:>6.2}, Val Acc: {4:>7.2%}, Time: {5} {6}'
                print(msg.format(total_batch, loss_train, acc_train, loss_val, acc_val, time_dif, improved_str))

            feed_dict[model.keep_prob] = config.dropout_keep_prob
            session.run(model.optim, feed_dict=feed_dict)  # 运行优化
            total_batch += 1

            if total_batch - last_improved > require_improvement:
                # 验证集正确率长期不提升，提前结束训练
                print("No optimization for a long time, auto-stopping...")
                flag = True
                break  # 跳出循环
        if flag:  # 同上
            break


def test():
    print("Loading test data...")
    start_time = time.time()
    x_test, y_test = process_file(test_dir, word_to_id, cat_to_id, config.seq_length)

    session = tf.Session()
    session.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.restore(sess=session, save_path=save_path)  # 读取保存的模型

    print('Testing...')
    loss_test, acc_test = evaluate(session, x_test, y_test)
    msg = 'Test Loss: {0:>6.2}, Test Acc: {1:>7.2%}'
    print(msg.format(loss_test, acc_test))

    batch_size = 128
    data_len = len(x_test)
    num_batch = int((data_len - 1) / batch_size) + 1

    y_test_cls = np.argmax(y_test, 1)
    y_pred_cls = np.zeros(shape=len(x_test), dtype=np.int32)  # 保存预测结果
    for i in range(num_batch):  # 逐批次处理
        start_id = i * batch_size
        end_id = min((i + 1) * batch_size, data_len)
        feed_dict = {
            model.input_x: x_test[start_id:end_id],
            model.keep_prob: 1.0
        }
        y_pred_cls[start_id:end_id] = session.run(model.y_pred_cls, feed_dict=feed_dict)

    # 评估
    print("Precision, Recall and F1-Score...")
    print(metrics.classification_report(y_test_cls, y_pred_cls, target_names=categories))

    # 混淆矩阵
    print("Confusion Matrix...")
    cm = metrics.confusion_matrix(y_test_cls, y_pred_cls)
    print(cm)

    time_dif = get_time_dif(start_time)
    print("Time usage:", time_dif)


if __name__ == '__main__':
    if len(sys.argv) != 2 or sys.argv[1] not in ['train', 'test']:
        raise ValueError("""usage: python run_cnn.py [train / test]""")

    print('Configuring CNN model...')
    config = TCNNConfig()
    if not os.path.exists(vocab_dir):  # 如果不存在词汇表，重建
        build_vocab(train_dir, vocab_dir, config.vocab_size)
    categories, cat_to_id = read_category()
    words, word_to_id = read_vocab(vocab_dir)
    config.vocab_size = len(words)
    model = TextCNN(config)

    if sys.argv[1] == 'train':
        train()
    else:
        test()


ModuleNotFoundError: No module named 'cnn_model'

In [4]:
class TesxtCNN(object):
    """CNN for text classification .
    Use an embedding layer, followed by a convolutional layer, 
    max-pooling layer and a softmax layer"""
    def __init__(self, sequence_length, num_classes, vocab_size, embedding_size, filter_sizes, num_filters):
        # placeholders for input, output and dropout
        self.input_x = tf.placeholder(dtype=tf.int32, shape=[None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(dtype=tf.float32, shape = [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(dtype=tf.float32, name="dropout_keep_prob")
    
        # define the embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            # w is the embedding matrix thar we learn during training and use randomuniform distribution to initialize it
            W = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
            name="W")
            self.embedded_chars = tf.nn.embedding_lookup(W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # define convlutional layer and maxpooling layer
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool" % filter_size):
                filter_shape=[filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.trucated_nomal(filter_shape, stddev=0.1), name = "W")
                b = tf.Variable(tf.constant(0.1, shape = [num_filters]), anme = "b")
                conv = tf.nn.con2d(self.embedded_chars_expanded,
                                  W,
                                  strides = [1, 1, 1, 1], # all 1 in four channels
                                  padding = "VALID", # means the dimension will change after conv
                                  name = "conv")
                # apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name = "relu")

                ##########above is conv
                # mxpooling over the outputs
                pooled = tf.nn.max_pool(
                h,
                ksize=[1, sequence_length - filter_size + 1, 1, 1],
                strides = [1,1,1,1],
                padding = "VALID",
                name = "pool")
                pooled_outputs.append(pooled)
    # Performing max-pooling over the output of a specific filter 
    # size leaves us with a tensor of shape 
    # [batch_size, 1, 1, num_filters]. 
    # [the size of batch(num of sentence in a batch),choose the max one , keep ,channels]

        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(3, pooled_outputs) # 最后一维进行拼接
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        with tf.name_scope("output"):
            W = tf.Variable(tf.trucated_normal([num_filters_total, num_classes], staddev = 0.1), name = "W")
            b = tf.Variable(tf.constant(0.1, shape = [num_classes]), name = "b")
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="score")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # cal mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(self.scores, self.input_y)
            self.loss = tf.reduce_mean(losses)

        # calculate accuaray
        with tf.name_scope("accuaracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name = "accuracy")



    
        
    
        

In [5]:
# skip tensorboard maybe later review it

In [6]:
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
    allow_soft_placement = FLAGS.allow_soft_platment,
    log_device_placement = FLAGS.log_device_platment)
    sess = tf.Session(config = session_conf)
    with sess.as_default():
        cnn = TextCNN(
        sequence_length = x_train.shape[1],
        num_classes = 2,
        vocab_size = len(vocabulary),
        embedding_size=FLAGS.embedding_dim,
        filter_sizes = map(int, FLAGS.filter_sizes.split(",")),
        num_filters = FLAGS.num_filters)
        
        # define optimize network's loss function using adam optimizer
        
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-4)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        
        
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.scalar_summary("loss", cnn.loss)
        acc_summary = tf.scalar_summary("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.merge_summary([loss_summary, acc_summary])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.train.SummaryWriter(train_summary_dir, sess.graph_def)

        # Dev summaries
        dev_summary_op = tf.merge_summary([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.train.SummaryWriter(dev_summary_dir, sess.graph_def)

NameError: name 'FLAGS' is not defined

In [7]:
# Checkpointing
checkpoint_dir = os.path.abspath(os.path.join(out_dir, &quot;checkpoints&quot;))
checkpoint_prefix = os.path.join(checkpoint_dir, &quot;model&quot;)
# Tensorflow assumes this directory already exists so we need to create it
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
saver = tf.train.Saver(tf.all_variables())

SyntaxError: invalid syntax (<ipython-input-7-f9b9910bc510>, line 2)

In [8]:
sess.run(tf.initialize_all_variables())


NameError: name 'sess' is not defined

In [9]:
def train_step(x_batch, y_batch):
    """
    A single training step
    """
    feed_dict = {
      cnn.input_x: x_batch,
      cnn.input_y: y_batch,
      cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
    }
    _, step, summaries, loss, accuracy = sess.run(
        [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
        feed_dict)
    time_str = datetime.datetime.now().isoformat()
    print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
    train_summary_writer.add_summary(summaries, step)

In [ ]:
def dev_step(x_batch, y_batch, writer=None):
    """
    Evaluates model on a dev set
    """
    feed_dict = {
      cnn.input_x: x_batch,
      cnn.input_y: y_batch,
      cnn.dropout_keep_prob: 1.0
    }
    step, summaries, loss, accuracy = sess.run(
        [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
        feed_dict)
    time_str = datetime.datetime.now().isoformat()
    print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
    if writer:
        writer.add_summary(summaries, step)